<a href="https://colab.research.google.com/github/ChoSanghyuk/EncoreProject/blob/master/Naver_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests

ModuleNotFoundError: ignored

In [ ]:
# 조건에 맞는 네이버 기사만 스크랩
def crawler(maxpage,query,s_date,e_date):
    s_from = s_date.replace(".","")     # url에 들어갈 시작 날짜
    e_to = e_date.replace(".","")       # url에 들어갈 끝 날짜
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1    # 네이버 형식에 맞게 수치변환 11= 2페이지 21=3페이지 31=4페이지 ...81=9페이지 , 91=10페이지, 101=11페이지
    f = open("C:/Users/Playdata/Documents/moonju/contents_text.csv", 'w', encoding='utf-8')     # 결과 저장할 파일 설정
    while page < maxpage_t:             
        print(page)
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=0&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        req = requests.get(url)         # url 주소 페이지 가져옴
        #print(url)
        cont = req.content              
        soup = BeautifulSoup(cont, 'html.parser')   # html 파싱
        #print(soup)
        for urls in soup.select("._sp_each_url"):   # 여러 언론사중 네이버 기사만 크롬창을 띄워 데이터 확인
            try :
                #print(urls["href"])
                if urls["href"].startswith("https://news.naver.com"):
                    #print(urls["href"])
                    news_detail = get_news(f,urls["href"])  # 크롬창에 기사 띄워 이후 작업
            except Exception as e:                  # 크롤링 도중 에러 시 에러 메시지 출력
                print(e)
                continue
        page += 10
    f.close()

In [ ]:

def get_news(f,url):

    #웹 드라이버 (캐시데이터 등을 초기화하기 위해 크롬 창을 띄움)
    driver = webdriver.Chrome('./chromedriver.exe')
    driver.implicitly_wait(5)   # 크롬창을 띄울때 시간차를 줘 오류 방지
    driver.get(url)

    # 클린봇 옵션 해제 후 추출(잘 안됨)
    # cleanbot  = driver.find_element_by_css_selector('a.u_cbox_cleanbot_setbutton')
    # cleanbot.click()
    # time.sleep(1)
    # cleanbot_disable = driver.find_element_by_xpath("//input[@id='cleanbot_dialog_checkbox_cbox_module']")
    # cleanbot_disable.click()
    # time.sleep(1)
    # cleanbot_confirm = driver.find_element_by_css_selector('div.u_cbox_layer_cleanbot_extrabutton')
    # cleanbot_confirm.click()
    # time.sleep(1)

    #더보기 계속 클릭하기
    while True:
        try:
            btn_more = driver.find_element_by_css_selector('a.u_cbox_btn_more') # 더보기 태그 
            btn_more.click()        # 클릭 액션
            # time.sleep(1)
        except:
            break

    #날짜 추출
    news_date = driver.find_element_by_css_selector('span.t11') # 해당 태그 하나만 선택해 추출
    news_date = news_date.text[:11]         # 날짜 슬라이싱

    #기사제목 추출
    article_head = driver.find_elements_by_css_selector('#articleTitle')
    # print("기사 제목 : " + article_head[0].text)

    # 좋아요 수 추출
    like_count = driver.find_elements_by_css_selector('em.u_cbox_cnt_recomm ')  # 더보기 펼쳐진 상태에서 좋아요 수 전부 추출하여 리스트 형식으로 저장

    # 남여 성비와 연령대 추출 (정상 작동)
    # per = driver.find_elements_by_css_selector('span.u_cbox_chart_per')
    # print("남자 성비 : " + per[0].text)
    # print("여자 성비 : " + per[1].text)
    # print("10대 : " + per[2].text)
    # print("20대 : " + per[3].text)
    # print("30대 : " + per[4].text)
    # print("40대 : " + per[5].text)
    # print("50대 : " + per[6].text)
    # print("60대 이상 : " + per[7].text)

    # 댓글추출
    contents = driver.find_elements_by_css_selector('span.u_cbox_contents') # 더보기로 펼쳐진 댓글 전부 추출하여 리스트 형식으로 저장
    # for content in contents:
    #     print(content.text)

    #좋아요 수 10개 미만 제거
    like = []
    cont = []
    for i in range(len(like_count)):
        if int(like_count[i].text) >= 10 :  # 좋아요 수가 10 이상인 것들 선택
            like.append(like_count[i].text) # 좋아요 데이터를 새로운 리스트에 삽입
            cont.append(contents[i].text)   # 댓글 데이터 새로운 리스트에 삽입

    for i in range(0, len(like)):           # 날짜, 제목, 댓글, 좋아요 순으로 csv 파일에 write
        f.write("{}, {}, {}, {}\n".format(news_date, article_head[0].text, cont[i], like[i]))


In [ ]:
def main():
    maxpage = "2"               # 검색할 페이지 수
    query = "공공 의대 확대"    # 키워드
    s_date = "2020.08.07"       # 검색 시작 날짜
    e_date = "2020.08.09"       # 검색 끝 날짜
    crawler(maxpage,query,s_date,e_date) # 네이버 뉴스 검색 창에서 크롤링 시작

In [ ]:
main()